# Bank Customers Churn
https://www.kaggle.com/datasets/santoshd3/bank-customers?resource=download

### Data Preprocessing

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# importing dataset
churn = pd.read_csv("Churn Modeling.csv")

In [3]:
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# reconstructing dataset by removing the irrelevant columns such as RowNumber, CustomerId, Surname
X = churn.iloc[:, 3:13]
y = churn.iloc[:, 13]

In [5]:
# converting categorical features into dummy variables
states = pd.get_dummies(X['Geography'],drop_first = True)
gender = pd.get_dummies(X['Gender'],drop_first = True)

In [6]:
# dropping the columns which were converted to dummy variables
X = X.drop(["Geography","Gender"],axis = 1)

In [7]:
# concatenating remaining columns of the dataset with the dummy variables columns
X = pd.concat([X,states,gender],axis = 1)

In [8]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [9]:
# splitting dataset into train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [10]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

### ANN

In [11]:
# importing keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

In [12]:
# initializing ANN
classifier = Sequential()

2022-06-28 14:59:57.855119: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# adding input layer and first hidden layer
classifier.add(Dense(activation = "relu", input_dim = 11, units = 6, kernel_initializer = "uniform"))

In [14]:
# adding second hidden layer
classifier.add(Dense(activation = "relu", units = 6, kernel_initializer = "uniform"))

In [15]:
# adding output layer
classifier.add(Dense(activation = "sigmoid", units = 1, kernel_initializer = "uniform"))

In [16]:
# compiling ANN
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [19]:
# fitting ANN to training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 50)

Epoch 1/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4819 - accuracy: 0.7958
Epoch 2/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4264 - accuracy: 0.7960
Epoch 3/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4205 - accuracy: 0.8064
Epoch 4/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4172 - accuracy: 0.8276
Epoch 5/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4155 - accuracy: 0.8291
Epoch 6/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4139 - accuracy: 0.8316
Epoch 7/50
800/800 [==============================] - 2s 2ms/step - loss: 0.4123 - accuracy: 0.8332
Epoch 8/50
800/800 [==============================] - 1s 2ms/step - loss: 0.4109 - accuracy: 0.8332
Epoch 9/50
800/800 [==============================] - 1s 2ms/step - loss: 0.4107 - accuracy: 0.8339
Epoch 10/50
800/800 [==============================] - 1s 2ms/step - loss: 0.4096 - accuracy: 0.8350

### Making prediction and Evaluating model

In [20]:
# predicting test set
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [21]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [22]:
# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [23]:
cm

array([[1541,   54],
       [ 264,  141]])

In [24]:
accuracy

0.841